In [148]:
import pandas as pd
import numpy as np

In [149]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
# import torch.nn as nn
# from torch.utils.data import DataLoader
# import torch.optim as optim
# import torch
from sklearn.model_selection import train_test_split
import os
import sys
from sklearn.metrics import accuracy_score
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report


In [150]:
# Set the root directory of your repo as part of the path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [151]:
data = pd.read_csv('../Data/combined/full_lathan.csv')

/var/folders/sj/xc1pvfqd479gs7_4_rxlkx_c0000gn/T/ipykernel_27631/4119657318.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Data/combined/full_lathan.csv')


In [152]:
vegas.columns

Index(['GameId', 'h_Date', 'h_Team', 'h_OppTeam', 'h_TeamId',
       'h_PercentBet_ML', 'h_Open_Line_ML', 'h_Pinnacle_ML', 'h_5dimes_ML',
       'h_Heritage_ML',
       ...
       'a_Best_Odds_OU', 'a_Worst_Odds_OU', 'a_Pts', 'a_Spread', 'a_Result',
       'a_YEAR', 'Total', 'Worst_Line_OU', 'Best_Line_OU', 'Average_Line_OU'],
      dtype='object', length=115)

In [153]:
# Vegas data
vegas = pd.read_csv('../Data/combined/full_vegas_data.csv')
vegas.head()

,Date,Location,Team,OppTeam,TeamId,GameId,PercentBet_ML,Open_Line_ML,Pinnacle_ML,5dimes_ML,Heritage_ML,Bovada_ML,Betonline_ML,Average_Line_ML,Best_Line_ML,Worst_Line_ML,PercentBet_Spread,Open_Line_Spread,Open_Odds_Spread,Pinnacle_Line_Spread,Pinnacle_Odds_Spread,5dimes_Line_Spread,5dimes_Odds_Spread,Heritage_Line_Spread,Heritage_Odds_Spread,Bovada_Line_Spread,Bovada_Odds_Spread,Betonline_Line_Spread,Betonline_Odds_Spread,Average_Line_Spread,Average_Odds_Spread,Best_Line_Spread,Worst_Line_Spread,Best_Odds_Spread,Worst_Odds_Spread,PercentBet_OU,Open_Line_OU,Open_Odds_OU,Pinnacle_Line_OU,Pinnacle_Odds_OU,5dimes_Line_OU,5dimes_Odds_OU,Heritage_Line_OU,Heritage_Odds_OU,Bovada_Line_OU,Bovada_Odds_OU,Betonline_Line_OU,Betonline_Odds_OU,Average_Line_OU,Average_Odds_OU,Best_Line_OU,Worst_Line_OU,Best_Odds_OU,Worst_Odds_OU,Pts,Spread,Result,Total,YEAR
0,2012-10-30,away,Washington,Cleveland,1610612764,21200001,44.6,220.0,210.0,200.0,210.0,215.0,210.0,209.0,215.0,200.0,44.4,5.0,-110.0,6.0,-105.0,6.0,-110.0,6.0,-110.0,6.0,-105.0,6.0,-108.0,5.83,-108.00,6.0,5.0,-105.0,-110.0,47.3,187.0,-110.0,191.0,-104.0,191.0,-110.0,191.0,-110.0,191.0,-105.0,191.0,-110.0,190.33,-108.17,191.0,187.0,-104.0,-110.0,84.0,-10.0,L,178.0,2012-2013
1,2012-10-30,home,Cleveland,Washington,1610612739,21200001,55.4,-260.0,-235.0,-240.0,-250.0,-255.0,-250.0,-246.0,-235.0,-255.0,55.6,-5.0,-110.0,-6.0,-105.0,-6.0,-110.0,-6.0,-110.0,-6.0,-115.0,-6.0,-112.0,-5.83,-110.33,-5.0,-6.0,-105.0,-115.0,52.7,187.0,-110.0,191.0,-106.0,191.0,-110.0,191.0,-110.0,191.0,-115.0,191.0,-110.0,190.33,-110.17,191.0,187.0,-106.0,-115.0,94.0,10.0,W,178.0,2012-2013
2,2012-10-30,away,Boston,Miami,1610612738,21200002,44.8,220.0,244.0,240.0,230.0,225.0,235.0,234.8,244.0,225.0,48.2,7.5,-110.0,6.5,-105.0,6.0,-101.0,6.0,-110.0,7.0,-115.0,6.5,-110.0,6.58,-108.50,7.5,6.0,-101.0,-115.0,51.6,182.0,-110.0,187.5,-105.0,187.5,-104.0,187.5,-110.0,188.0,-105.0,187.0,-110.0,186.58,-107.33,188.0,182.0,-104.0,-110.0,107.0,-13.0,L,227.0,2012-2013
3,2012-10-30,home,Miami,Boston,1610612748,21200002,55.2,-260.0,-275.0,-260.0,-270.0,-265.0,-280.0,-270.0,-260.0,-280.0,51.8,-7.5,-110.0,-6.5,-105.0,-6.0,-109.0,-6.0,-110.0,-7.0,-105.0,-6.5,-110.0,-6.58,-108.17,-6.0,-7.5,-105.0,-110.0,48.4,182.0,-110.0,187.5,-105.0,187.5,-106.0,187.5,-110.0,188.0,-115.0,187.0,-110.0,186.58,-109.33,188.0,182.0,-105.0,-115.0,120.0,13.0,W,227.0,2012-2013
4,2012-10-30,away,Dallas,L.A. Lakers,1610612742,21200003,40.8,350.0,393.0,420.0,400.0,375.0,385.0,394.6,420.0,375.0,49.0,9.5,-110.0,8.5,-103.0,8.5,-105.0,8.5,-110.0,9.0,-105.0,8.5,-110.0,8.75,-107.17,9.5,8.5,-103.0,-110.0,42.6,185.0,-110.0,186.0,-100.0,186.0,-105.0,185.5,-110.0,187.0,-110.0,186.5,-110.0,186.00,-107.50,187.0,185.0,-100.0,-110.0,99.0,8.0,W,190.0,2012-2013


In [154]:
# sort vegas by GameID then Location
vegas = vegas.sort_values(by=['GameId', 'Location'])
vegas

,Date,Location,Team,OppTeam,TeamId,GameId,PercentBet_ML,Open_Line_ML,Pinnacle_ML,5dimes_ML,Heritage_ML,Bovada_ML,Betonline_ML,Average_Line_ML,Best_Line_ML,Worst_Line_ML,PercentBet_Spread,Open_Line_Spread,Open_Odds_Spread,Pinnacle_Line_Spread,Pinnacle_Odds_Spread,5dimes_Line_Spread,5dimes_Odds_Spread,Heritage_Line_Spread,Heritage_Odds_Spread,Bovada_Line_Spread,Bovada_Odds_Spread,Betonline_Line_Spread,Betonline_Odds_Spread,Average_Line_Spread,Average_Odds_Spread,Best_Line_Spread,Worst_Line_Spread,Best_Odds_Spread,Worst_Odds_Spread,PercentBet_OU,Open_Line_OU,Open_Odds_OU,Pinnacle_Line_OU,Pinnacle_Odds_OU,5dimes_Line_OU,5dimes_Odds_OU,Heritage_Line_OU,Heritage_Odds_OU,Bovada_Line_OU,Bovada_Odds_OU,Betonline_Line_OU,Betonline_Odds_OU,Average_Line_OU,Average_Odds_OU,Best_Line_OU,Worst_Line_OU,Best_Odds_OU,Worst_Odds_OU,Pts,Spread,Result,Total,YEAR
0,2012-10-30,away,Washington,Cleveland,1610612764,21200001,44.6,220.0,210.0,200.0,210.0,215.0,210.0,209.0,215.0,200.0,44.4,5.0,-110.0,6.0,-105.0,6.0,-110.0,6.0,-110.0,6.0,-105.0,6.0,-108.0,5.83,-108.00,6.0,5.0,-105.0,-110.0,47.3,187.0,-110.0,191.0,-104.0,191.0,-110.0,191.0,-110.0,191.0,-105.0,191.0,-110.0,190.33,-108.17,191.0,187.0,-104.0,-110.0,84.0,-10.0,L,178.0,2012-2013
1,2012-10-30,home,Cleveland,Washington,1610612739,21200001,55.4,-260.0,-235.0,-240.0,-250.0,-255.0,-250.0,-246.0,-235.0,-255.0,55.6,-5.0,-110.0,-6.0,-105.0,-6.0,-110.0,-6.0,-110.0,-6.0,-115.0,-6.0,-112.0,-5.83,-110.33,-5.0,-6.0,-105.0,-115.0,52.7,187.0,-110.0,191.0,-106.0,191.0,-110.0,191.0,-110.0,191.0,-115.0,191.0,-110.0,190.33,-110.17,191.0,187.0,-106.0,-115.0,94.0,10.0,W,178.0,2012-2013
2,2012-10-30,away,Boston,Miami,1610612738,21200002,44.8,220.0,244.0,240.0,230.0,225.0,235.0,234.8,244.0,225.0,48.2,7.5,-110.0,6.5,-105.0,6.0,-101.0,6.0,-110.0,7.0,-115.0,6.5,-110.0,6.58,-108.50,7.5,6.0,-101.0,-115.0,51.6,182.0,-110.0,187.5,-105.0,187.5,-104.0,187.5,-110.0,188.0,-105.0,187.0,-110.0,186.58,-107.33,188.0,182.0,-104.0,-110.0,107.0,-13.0,L,227.0,2012-2013
3,2012-10-30,home,Miami,Boston,1610612748,21200002,55.2,-260.0,-275.0,-260.0,-270.0,-265.0,-280.0,-270.0,-260.0,-280.0,51.8,-7.5,-110.0,-6.5,-105.0,-6.0,-109.0,-6.0,-110.0,-7.0,-105.0,-6.5,-110.0,-6.58,-108.17,-6.0,-7.5,-105.0,-110.0,48.4,182.0,-110.0,187.5,-105.0,187.5,-106.0,187.5,-110.0,188.0,-115.0,187.0,-110.0,186.58,-109.33,188.0,182.0,-105.0,-115.0,120.0,13.0,W,227.0,2012-2013
4,2012-10-30,away,Dallas,L.A. Lakers,1610612742,21200003,40.8,350.0,393.0,420.0,400.0,375.0,385.0,394.6,420.0,375.0,49.0,9.5,-110.0,8.5,-103.0,8.5,-105.0,8.5,-110.0,9.0,-105.0,8.5,-110.0,8.75,-107.17,9.5,8.5,-103.0,-110.0,42.6,185.0,-110.0,186.0,-100.0,186.0,-105.0,185.5,-110.0,187.0,-110.0,186.5,-110.0,186.00,-107.50,187.0,185.0,-100.0,-110.0,99.0,8.0,W,190.0,2012-2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17203,2019-04-10,home,Denver,Minnesota,1610612743,21801228,45.2,-900.0,-794.0,-830.0,-920.0,-1000.0,-900.0,-888.8,-794.0,-1000.0,59.6,-12.0,-110.0,-12.5,-106.0,-12.5,-105.0,-13.0,-105.0,-13.0,-110.0,-13.0,-110.0,-12.67,-107.67,-12.0,-13.0,-105.0,-110.0,43.8,218.5,-110.0,218.5,-105.0,218.5,-108.0,218.5,-105.0,218.0,-105.0,218.5,-110.0,218.42,-107.17,218.5,218.0,-105.0,-110.0,99.0,4.0,W,194.0,2018-2019
17204,2019-04-10,away,Utah,L.A. Clippers,1610612762,21801229,61.6,199.0,249.0,275.0,270.0,270.0,250.0,262.8,275.0,249.0,39.7,4.0,-110.0,7.5,-103.0,8.0,-105.0,8.0,-105.0,8.5,-115.0,8.0,-114.0,7.33,-108.67,8.5,4.0,-103.0,-115.0,46.5,NaN,NaN,226.0,-105.0,225.5,-102.0,225.5,-105.0,225.0,-115.0,225.5,-110.0,225.50,-107.40,226.0,225.0,-102.0,-115.0,137.0,-6.0,L,280.0,2018-2019
17205,2019-04-10,home,L.A. Clippers,Utah,1610612746,21801229,38.4,-240.0,-281.0,-305.0,-330.0,-340.0,-300.0,-311.2,-281.0,-340.0,60.3,-4.0,-133.0,-7.5,-107.0,-8.0,-105.0,-8.0,-105.0,-8.5,-105.0,-8.0,-106.0,-7.33,-110.17,-4.0,-8

In [155]:
vegas.columns

Index(['Date', 'Location', 'Team', 'OppTeam', 'TeamId', 'GameId',
       'PercentBet_ML', 'Open_Line_ML', 'Pinnacle_ML', '5dimes_ML',
       'Heritage_ML', 'Bovada_ML', 'Betonline_ML', 'Average_Line_ML',
       'Best_Line_ML', 'Worst_Line_ML', 'PercentBet_Spread',
       'Open_Line_Spread', 'Open_Odds_Spread', 'Pinnacle_Line_Spread',
       'Pinnacle_Odds_Spread', '5dimes_Line_Spread', '5dimes_Odds_Spread',
       'Heritage_Line_Spread', 'Heritage_Odds_Spread', 'Bovada_Line_Spread',
       'Bovada_Odds_Spread', 'Betonline_Line_Spread', 'Betonline_Odds_Spread',
       'Average_Line_Spread', 'Average_Odds_Spread', 'Best_Line_Spread',
       'Worst_Line_Spread', 'Best_Odds_Spread', 'Worst_Odds_Spread',
       'PercentBet_OU', 'Open_Line_OU', 'Open_Odds_OU', 'Pinnacle_Line_OU',
       'Pinnacle_Odds_OU', '5dimes_Line_OU', '5dimes_Odds_OU',
       'Heritage_Line_OU', 'Heritage_Odds_OU', 'Bovada_Line_OU',
       'Bovada_Odds_OU', 'Betonline_Line_OU', 'Betonline_Odds_OU',
       'Average_Line

In [156]:
# --- Step 1: Define shared columns (same across home/away teams) ---
shared_columns = ['Total', 'Worst_Line_OU', 'Best_Line_OU', 'Average_Line_OU']

# --- Step 2: Split into home and away teams ---
df_home = vegas[vegas['Location'] == 'home'].copy()
df_away = vegas[vegas['Location'] == 'away'].copy()

# --- Step 3: Drop shared columns from away team to avoid duplication ---
df_home = df_home.drop(columns=['Location'])  # Don't need location after naming
df_away = df_away.drop(columns=shared_columns + ['Location'])

# --- Step 4: Prefix all columns (except identifiers) ---
home_prefix = 'h_'
away_prefix = 'a_'
id_cols = ['GameId']  # Columns to merge on

df_home = df_home.rename(columns={col: home_prefix + col for col in df_home.columns if col not in id_cols})
df_away = df_away.rename(columns={col: away_prefix + col for col in df_away.columns if col not in id_cols})

# --- Step 5: Merge home and away teams on GameId ---
df_merged = pd.merge(df_home, df_away, on='GameId', how='inner')

# --- Step 6: Add shared columns (only from home, since they're the same) ---
shared_data = vegas[vegas['Location'] == 'home'][['GameId'] + shared_columns]
df_final = pd.merge(df_merged, shared_data, on='GameId', how='left')

# Optional: reorder columns to start with GameId
cols = ['GameId'] + [col for col in df_final.columns if col != 'GameId']
df_final = df_final[cols]

In [157]:
vegas = df_final

In [158]:
from src.data.preprocess import (
    combine_team_rows,
    add_home_win_column,
    split_wins_losses,
    add_playoff_indicator,
    combine_date_columns,
    remove_all_star_and_playoff_games,
    create_ml_dataset

)

In [159]:
# Creating key predictor columns
data2 = create_ml_dataset(data)

In [160]:
# Dropping GAME_DATE_EST
data2 = data2.drop(columns=['GAME_DATE_EST'])

In [161]:
data2.columns

Index(['GAME_ID', 'GAME_SEQUENCE', 'Date', 'YEAR', 'a_index', 'a_TEAM_ID',
       'a_TEAM_WINS_LOSSES', 'a_PTS', 'a_FG_PCT', 'a_FT_PCT', 'a_FG3_PCT',
       'a_AST', 'a_REB', 'a_TOV', 'h_index', 'h_TEAM_ID', 'h_TEAM_WINS_LOSSES',
       'h_PTS', 'h_FG_PCT', 'h_FT_PCT', 'h_FG3_PCT', 'h_AST', 'h_REB', 'h_TOV',
       'HOME_WIN', 'a_WINS', 'a_LOSSES', 'h_WINS', 'h_LOSSES',
       'a_SEASON_GAMES_PLAYED', 'h_SEASON_GAMES_PLAYED', 'IS_PLAYOFF_GAME',
       'a_season_PTS_lag', 'a_season_AST_lag', 'a_season_REB_lag',
       'a_season_TOV_lag', 'a_season_FG_PCT_avg', 'a_season_FT_PCT_avg',
       'a_season_FG3_PCT_avg', 'a_WIN_PCT', 'a_WIN_PCT_LAST10',
       'h_season_PTS_lag', 'h_season_AST_lag', 'h_season_REB_lag',
       'h_season_TOV_lag', 'h_season_FG_PCT_avg', 'h_season_FT_PCT_avg',
       'h_season_FG3_PCT_avg', 'h_WIN_PCT', 'h_WIN_PCT_LAST10'],
      dtype='object')

In [162]:
vegas.shape, data2.shape

((8604, 115), (8610, 50))

In [163]:
combined_df = pd.merge(data2, vegas, left_on = 'GAME_ID',right_on='GameId', how='left')

In [164]:
# Drop lines from specific books. Keep only best, worst, and average lines
# --- Step 1: Define sportsbook names to remove ---
books_to_drop = ['Pinnacle', '5dimes', 'Heritage', 'Bovada', 'Betonline']

# --- Step 2: Identify columns to drop ---
columns_to_drop = [col for col in combined_df.columns if any(book in col for book in books_to_drop)]

# --- Step 3: Drop those columns from the DataFrame ---
combined_df = combined_df.drop(columns=columns_to_drop)
columns_to_drop = [col for col in combined_df.columns if any(book in col for book in books_to_drop)]
combined_df = combined_df.drop(columns=columns_to_drop)


In [165]:
combined_df

,GAME_ID,GAME_SEQUENCE,Date,YEAR,a_index,a_TEAM_ID,a_TEAM_WINS_LOSSES,a_PTS,a_FG_PCT,a_FT_PCT,a_FG3_PCT,a_AST,a_REB,a_TOV,h_index,h_TEAM_ID,h_TEAM_WINS_LOSSES,h_PTS,h_FG_PCT,h_FT_PCT,h_FG3_PCT,h_AST,h_REB,h_TOV,HOME_WIN,a_WINS,a_LOSSES,h_WINS,h_LOSSES,a_SEASON_GAMES_PLAYED,h_SEASON_GAMES_PLAYED,IS_PLAYOFF_GAME,a_season_PTS_lag,a_season_AST_lag,a_season_REB_lag,a_season_TOV_lag,a_season_FG_PCT_avg,a_season_FT_PCT_avg,a_season_FG3_PCT_avg,a_WIN_PCT,a_WIN_PCT_LAST10,h_season_PTS_lag,h_season_AST_lag,h_season_REB_lag,h_season_TOV_lag,h_season_FG_PCT_avg,h_season_FT_PCT_avg,h_season_FG3_PCT_avg,h_WIN_PCT,h_WIN_PCT_LAST10,GameId,h_Date,h_Team,h_OppTeam,h_TeamId,h_PercentBet_ML,h_Open_Line_ML,h_Average_Line_ML,h_Best_Line_ML,h_Worst_Line_ML,h_PercentBet_Spread,h_Open_Line_Spread,h_Open_Odds_Spread,h_Average_Line_Spread,h_Average_Odds_Spread,h_Best_Line_Spread,h_Worst_Line_Spread,h_Best_Odds_Spread,h_Worst_Odds_Spread,h_PercentBet_OU,h_Open_Line_OU,h_Open_Odds_OU,h_Average_Line_OU,h_Average_Odds_OU,h_Best_Line_OU,h_Worst_Line_OU,h_Best_Odds_OU,h_Worst_Odds_OU,h_Pts,h_Spread,h_Result,h_Total,h_YEAR,a_Date,a_Team,a_OppTeam,a_TeamId,a_PercentBet_ML,a_Open_Line_ML,a_Average_Line_ML,a_Best_Line_ML,a_Worst_Line_ML,a_PercentBet_Spread,a_Open_Line_Spread,a_Open_Odds_Spread,a_Average_Line_Spread,a_Average_Odds_Spread,a_Best_Line_Spread,a_Worst_Line_Spread,a_Best_Odds_Spread,a_Worst_Odds_Spread,a_PercentBet_OU,a_Open_Line_OU,a_Open_Odds_OU,a_Average_Odds_OU,a_Best_Odds_OU,a_Worst_Odds_OU,a_Pts,a_Spread,a_Result,a_YEAR,Total,Worst_Line_OU,Best_Line_OU,Average_Line_OU
0,21200001,1,2012-10-30,2012-2013,1,1610612739,1-0,94,0.456,0.682,0.350,22,54,20,0,1610612764,0-1,84,0.356,0.600,0.250,26,39,12,0,0,0,0,0,0,0,0,0,0,0,0,0.465846,0.804308,0.361782,NaN,NaN,0,0,0,0,0.448566,0.772697,0.339461,NaN,NaN,21200001.0,2012-10-30,Cleveland,Washington,1.610613e+09,55.4,-260.0,-246.00,-235.0,-255.0,55.6,-5.0,-110.0,-5.83,-110.33,-5.0,-6.0,-105.0,-115.0,52.7,187.0,-110.0,190.33,-110.17,191.0,187.0,-106.0,-115.0,94.0,10.0,W,178.0,2012-2013,2012-10-30,Washington,Cleveland,1.610613e+09,44.6,220.0,209.00,215.0,200.0,44.4,5.0,-110.0,5.83,-108.00,6.0,5.0,-105.0,-110.0,47.3,187.0,-110.0,-108.17,-104.0,-110.0,84.0,-10.0,L,2012-2013,178.0,187.0,191.0,190.33
1,21200002,2,2012-10-30,2012-2013,2,1610612738,0-1,107,0.520,0.821,0.462,24,41,15,3,1610612748,1-0,120,0.544,0.813,0.500,25,36,8,1,0,0,0,0,0,0,0,0,0,0,0,0.451780,0.751817,0.350976,NaN,NaN,0,0,0,0,0.471600,0.683800,0.320200,NaN,NaN,21200002.0,2012-10-30,Miami,Boston,1.610613e+09,55.2,-260.0,-270.00,-260.0,-280.0,51.8,-7.5,-110.0,-6.58,-108.17,-6.0,-7.5,-105.0,-110.0,48.4,182.0,-110.0,186.58,-109.33,188.0,182.0,-105.0,-115.0,120.0,13.0,W,227.0,2012-2013,2012-10-30,Boston,Miami,1.610613e+09,44.8,220.0,234.80,244.0,225.0,48.2,7.5,-110.0,6.58,-108.50,7.5,6.0,-101.0,-115.0,51.6,182.0,-110.0,-107.33,-104.0,-110.0,107.0,-13.0,L,2012-2013,227.0,182.0,188.0,186.58
2,21200003,3,2012-10-30,2012-2013,4,1610612742,1-0,99,0.471,0.778,0.333,22,40,11,5,1610612747,0-1,91,0.494,0.387,0.231,24,46,14,0,0,0,0,0,0,0,0,0,0,0,0,0.452739,0.786101,0.344986,NaN,NaN,0,0,0,0,0.480692,0.803808,0.365885,NaN,NaN,21200003.0,2012-10-30,L.A. Lakers,Dallas,1.610613e+09,59.2,-425.0,-492.00,-460.0,-550.0,51.0,-9.5,-110.0,-8.75,-109.50,-8.5,-9.5,-105.0,-115.0,57.4,185.0,-110.0,186.00,-109.17,187.0,185.0,-105.0,-110.0,91.0,-8.0,L,190.0,2012-2013,2012-10-30,Dallas,L.A. Lakers,1.610613e+09,40.8,350.0,394.60,420.0,375.0,49.0,9.5,-110.0,8.75,-107.17,9.5,8.5,-103.0,-110.0,42.6,185.0,-110.0,-107.50,-100.0,-110.0,99.0,8.0,W,2012-2013,190.0,185.0,187.0,186.00
3,21200004,1,2012-10-31,2012-2013,6,1610612754,1-0,90,0.474,0.688,0.455,22,46,18,7,1610612761,0-1,88,0.363,0.842,0.353,18,42,10,0,0,0,0,0,0,0,0,0,0,0,0,0.456184,0.779842,0.359789,NaN,NaN,0,0,0,0,0.454478,0.714000,0.349896,NaN,NaN,21200004.0,2012-10-31,Toronto,Indiana,1.610613e+09,58.1,125.0,50.00,102.0,-102.0,48.5,4.0,-110.0,1.92,-110.50,4.0,1.0,-108.0,-115.0,46.5,188.0,-110.0,189.75,-108.50,190.5,188.0,-101.0,-1

In [166]:
columns_to_drop = ['a_FG_PCT', 'a_FT_PCT','a_FG3_PCT', 'a_AST', 'a_REB', 'a_TOV','GAME_ID', 'h_FG_PCT', 
                   'h_FT_PCT', 'h_FG3_PCT','h_AST', 'h_REB', 'h_TOV',
                   'GAME_SEQUENCE', 'Date', 'a_index', 'a_TEAM_ID','h_TEAM_ID', 'a_TEAM_WINS_LOSSES',
                  'h_TEAM_WINS_LOSSES', 'IS_PLAYOFF_GAME', 'h_index', 'HOME_WIN', 'a_SEASON_GAMES_PLAYED', 'h_SEASON_GAMES_PLAYED', 
                  'a_WINS', 'a_LOSSES', 'h_WINS', 'a_SEASON_GAMES_PLAYED', 'h_SEASON_GAMES_PLAYED', 'a_season_PTS_lag', 'a_season_AST_lag', 
                  'a_season_REB_lag', 'a_season_TOV_lag', 'h_season_PTS_lag', 'h_season_AST_lag', 'h_season_REB_lag', 'h_season_TOV_lag', 'GameId', 
                  'h_Date', 'h_Team', 'h_OppTeam', 'h_TeamId', 'h_YEAR', 'h_Result', 'h_Total', 'a_Date', 'a_Team', 'a_OppTeam', 'a_TeamId', 'a_YEAR', 'a_Result']
combined_df = combined_df.drop(columns=columns_to_drop)
combined_df

,YEAR,a_PTS,h_PTS,h_LOSSES,a_season_FG_PCT_avg,a_season_FT_PCT_avg,a_season_FG3_PCT_avg,a_WIN_PCT,a_WIN_PCT_LAST10,h_season_FG_PCT_avg,h_season_FT_PCT_avg,h_season_FG3_PCT_avg,h_WIN_PCT,h_WIN_PCT_LAST10,h_PercentBet_ML,h_Open_Line_ML,h_Average_Line_ML,h_Best_Line_ML,h_Worst_Line_ML,h_PercentBet_Spread,h_Open_Line_Spread,h_Open_Odds_Spread,h_Average_Line_Spread,h_Average_Odds_Spread,h_Best_Line_Spread,h_Worst_Line_Spread,h_Best_Odds_Spread,h_Worst_Odds_Spread,h_PercentBet_OU,h_Open_Line_OU,h_Open_Odds_OU,h_Average_Line_OU,h_Average_Odds_OU,h_Best_Line_OU,h_Worst_Line_OU,h_Best_Odds_OU,h_Worst_Odds_OU,h_Pts,h_Spread,a_PercentBet_ML,a_Open_Line_ML,a_Average_Line_ML,a_Best_Line_ML,a_Worst_Line_ML,a_PercentBet_Spread,a_Open_Line_Spread,a_Open_Odds_Spread,a_Average_Line_Spread,a_Average_Odds_Spread,a_Best_Line_Spread,a_Worst_Line_Spread,a_Best_Odds_Spread,a_Worst_Odds_Spread,a_PercentBet_OU,a_Open_Line_OU,a_Open_Odds_OU,a_Average_Odds_OU,a_Best_Odds_OU,a_Worst_Odds_OU,a_Pts,a_Spread,Total,Worst_Line_OU,Best_Line_OU,Average_Line_OU
0,2012-2013,94,84,0,0.465846,0.804308,0.361782,NaN,NaN,0.448566,0.772697,0.339461,NaN,NaN,55.4,-260.0,-246.00,-235.0,-255.0,55.6,-5.0,-110.0,-5.83,-110.33,-5.0,-6.0,-105.0,-115.0,52.7,187.0,-110.0,190.33,-110.17,191.0,187.0,-106.0,-115.0,94.0,10.0,44.6,220.0,209.00,215.0,200.0,44.4,5.0,-110.0,5.83,-108.00,6.0,5.0,-105.0,-110.0,47.3,187.0,-110.0,-108.17,-104.0,-110.0,84.0,-10.0,178.0,187.0,191.0,190.33
1,2012-2013,107,120,0,0.451780,0.751817,0.350976,NaN,NaN,0.471600,0.683800,0.320200,NaN,NaN,55.2,-260.0,-270.00,-260.0,-280.0,51.8,-7.5,-110.0,-6.58,-108.17,-6.0,-7.5,-105.0,-110.0,48.4,182.0,-110.0,186.58,-109.33,188.0,182.0,-105.0,-115.0,120.0,13.0,44.8,220.0,234.80,244.0,225.0,48.2,7.5,-110.0,6.58,-108.50,7.5,6.0,-101.0,-115.0,51.6,182.0,-110.0,-107.33,-104.0,-110.0,107.0,-13.0,227.0,182.0,188.0,186.58
2,2012-2013,99,91,0,0.452739,0.786101,0.344986,NaN,NaN,0.480692,0.803808,0.365885,NaN,NaN,59.2,-425.0,-492.00,-460.0,-550.0,51.0,-9.5,-110.0,-8.75,-109.50,-8.5,-9.5,-105.0,-115.0,57.4,185.0,-110.0,186.00,-109.17,187.0,185.0,-105.0,-110.0,91.0,-8.0,40.8,350.0,394.60,420.0,375.0,49.0,9.5,-110.0,8.75,-107.17,9.5,8.5,-103.0,-110.0,42.6,185.0,-110.0,-107.50,-100.0,-110.0,99.0,8.0,190.0,185.0,187.0,186.00
3,2012-2013,90,88,0,0.456184,0.779842,0.359789,NaN,NaN,0.454478,0.714000,0.349896,NaN,NaN,58.1,125.0,50.00,102.0,-102.0,48.5,4.0,-110.0,1.92,-110.50,4.0,1.0,-108.0,-115.0,46.5,188.0,-110.0,189.75,-108.50,190.5,188.0,-101.0,-110.0,88.0,-2.0,41.9,-145.0,-117.75,-113.0,-120.0,51.5,-4.0,-110.0,-1.92,-107.83,-1.0,-4.0,-102.0,-110.0,53.5,188.0,-110.0,-109.83,-109.0,-110.0,90.0,2.0,178.0,188.0,190.5,189.75
4,2012-2013,75,84,0,0.446300,0.740671,0.340429,NaN,NaN,0.477140,0.743980,0.374400,NaN,NaN,46.6,-125.0,109.75,110.0,109.0,41.6,-1.5,-110.0,1.25,-107.83,2.0,-1.5,-102.0,-110.0,52.9,202.0,-110.0,197.83,-110.00,202.0,196.5,-110.0,-110.0,84.0,9.0,53.4,105.0,-127.50,-120.0,-130.0,58.4,1.5,-110.0,-1.25,-110.50,1.5,-2.0,-108.0,-115.0,47.1,202.0,-110.0,-108.33,-100.0,-110.0,75.0,-9.0,159.0,196.5,202.0,197.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8605,2018-2019,116,127,33,0.476958,0.784458,0.358021,0.740741,0.5,0.454015,0.715500,0.348742,0.592593,0.5,43.8,148.0,176.20,181.0,175.0,37.7,2.0,-102.0,4.08,-105.83,4.5,2.0,-102.0,-110.0,57.6,230.0,-110.0,227.67,-106.17,230.0,227.0,-102.0,-110.0,116.0,-11.0,56.2,-170.0,-202.40,-190.0,-210.0,62.3,-2.0,-123.0,-4.08,-110.00,-2.0,-4.5,-101.0,-123.0,42.4,230.0,-110.0,-108.83,-105.0,-115.0,127.0,11.0,243.0,227.0,230.0,227.67
8606,2018-2019,94,105,34,0.446870,0.740536,0.341058,0.407407,0.5,0.477413,0.819921,0.392302,0.580247,0.6,41.7,-1600.0,-1238.60,-1113.0,-1400.0,44.3,-13.0,-110.0,-14.00,-105.17,-13.0,-14.5,-101.0,-110.0,50.4,218.0,-110.0,218.42,-109.17,219.0,218.0,-1

In [167]:
combined_df['h_cover'] = np.where((combined_df['h_PTS'] + combined_df['h_Average_Line_Spread'] - combined_df['a_PTS']) > 0, 1, 0)
combined_df['over_cover'] = np.where(combined_df['Total'] > combined_df['Average_Line_OU'], 1, 0)
combined_df

,YEAR,a_PTS,h_PTS,h_LOSSES,a_season_FG_PCT_avg,a_season_FT_PCT_avg,a_season_FG3_PCT_avg,a_WIN_PCT,a_WIN_PCT_LAST10,h_season_FG_PCT_avg,h_season_FT_PCT_avg,h_season_FG3_PCT_avg,h_WIN_PCT,h_WIN_PCT_LAST10,h_PercentBet_ML,h_Open_Line_ML,h_Average_Line_ML,h_Best_Line_ML,h_Worst_Line_ML,h_PercentBet_Spread,h_Open_Line_Spread,h_Open_Odds_Spread,h_Average_Line_Spread,h_Average_Odds_Spread,h_Best_Line_Spread,h_Worst_Line_Spread,h_Best_Odds_Spread,h_Worst_Odds_Spread,h_PercentBet_OU,h_Open_Line_OU,h_Open_Odds_OU,h_Average_Line_OU,h_Average_Odds_OU,h_Best_Line_OU,h_Worst_Line_OU,h_Best_Odds_OU,h_Worst_Odds_OU,h_Pts,h_Spread,a_PercentBet_ML,a_Open_Line_ML,a_Average_Line_ML,a_Best_Line_ML,a_Worst_Line_ML,a_PercentBet_Spread,a_Open_Line_Spread,a_Open_Odds_Spread,a_Average_Line_Spread,a_Average_Odds_Spread,a_Best_Line_Spread,a_Worst_Line_Spread,a_Best_Odds_Spread,a_Worst_Odds_Spread,a_PercentBet_OU,a_Open_Line_OU,a_Open_Odds_OU,a_Average_Odds_OU,a_Best_Odds_OU,a_Worst_Odds_OU,a_Pts,a_Spread,Total,Worst_Line_OU,Best_Line_OU,Average_Line_OU,h_cover,over_cover
0,2012-2013,94,84,0,0.465846,0.804308,0.361782,NaN,NaN,0.448566,0.772697,0.339461,NaN,NaN,55.4,-260.0,-246.00,-235.0,-255.0,55.6,-5.0,-110.0,-5.83,-110.33,-5.0,-6.0,-105.0,-115.0,52.7,187.0,-110.0,190.33,-110.17,191.0,187.0,-106.0,-115.0,94.0,10.0,44.6,220.0,209.00,215.0,200.0,44.4,5.0,-110.0,5.83,-108.00,6.0,5.0,-105.0,-110.0,47.3,187.0,-110.0,-108.17,-104.0,-110.0,84.0,-10.0,178.0,187.0,191.0,190.33,0,0
1,2012-2013,107,120,0,0.451780,0.751817,0.350976,NaN,NaN,0.471600,0.683800,0.320200,NaN,NaN,55.2,-260.0,-270.00,-260.0,-280.0,51.8,-7.5,-110.0,-6.58,-108.17,-6.0,-7.5,-105.0,-110.0,48.4,182.0,-110.0,186.58,-109.33,188.0,182.0,-105.0,-115.0,120.0,13.0,44.8,220.0,234.80,244.0,225.0,48.2,7.5,-110.0,6.58,-108.50,7.5,6.0,-101.0,-115.0,51.6,182.0,-110.0,-107.33,-104.0,-110.0,107.0,-13.0,227.0,182.0,188.0,186.58,1,1
2,2012-2013,99,91,0,0.452739,0.786101,0.344986,NaN,NaN,0.480692,0.803808,0.365885,NaN,NaN,59.2,-425.0,-492.00,-460.0,-550.0,51.0,-9.5,-110.0,-8.75,-109.50,-8.5,-9.5,-105.0,-115.0,57.4,185.0,-110.0,186.00,-109.17,187.0,185.0,-105.0,-110.0,91.0,-8.0,40.8,350.0,394.60,420.0,375.0,49.0,9.5,-110.0,8.75,-107.17,9.5,8.5,-103.0,-110.0,42.6,185.0,-110.0,-107.50,-100.0,-110.0,99.0,8.0,190.0,185.0,187.0,186.00,0,1
3,2012-2013,90,88,0,0.456184,0.779842,0.359789,NaN,NaN,0.454478,0.714000,0.349896,NaN,NaN,58.1,125.0,50.00,102.0,-102.0,48.5,4.0,-110.0,1.92,-110.50,4.0,1.0,-108.0,-115.0,46.5,188.0,-110.0,189.75,-108.50,190.5,188.0,-101.0,-110.0,88.0,-2.0,41.9,-145.0,-117.75,-113.0,-120.0,51.5,-4.0,-110.0,-1.92,-107.83,-1.0,-4.0,-102.0,-110.0,53.5,188.0,-110.0,-109.83,-109.0,-110.0,90.0,2.0,178.0,188.0,190.5,189.75,0,0
4,2012-2013,75,84,0,0.446300,0.740671,0.340429,NaN,NaN,0.477140,0.743980,0.374400,NaN,NaN,46.6,-125.0,109.75,110.0,109.0,41.6,-1.5,-110.0,1.25,-107.83,2.0,-1.5,-102.0,-110.0,52.9,202.0,-110.0,197.83,-110.00,202.0,196.5,-110.0,-110.0,84.0,9.0,53.4,105.0,-127.50,-120.0,-130.0,58.4,1.5,-110.0,-1.25,-110.50,1.5,-2.0,-108.0,-115.0,47.1,202.0,-110.0,-108.33,-100.0,-110.0,75.0,-9.0,159.0,196.5,202.0,197.83,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8605,2018-2019,116,127,33,0.476958,0.784458,0.358021,0.740741,0.5,0.454015,0.715500,0.348742,0.592593,0.5,43.8,148.0,176.20,181.0,175.0,37.7,2.0,-102.0,4.08,-105.83,4.5,2.0,-102.0,-110.0,57.6,230.0,-110.0,227.67,-106.17,230.0,227.0,-102.0,-110.0,116.0,-11.0,56.2,-170.0,-202.40,-190.0,-210.0,62.3,-2.0,-123.0,-4.08,-110.00,-2.0,-4.5,-101.0,-123.0,42.4,230.0,-110.0,-108.83,-105.0,-115.0,127.0,11.0,243.0,227.0,230.0,227.67,1,1
8606,2018-2019,94,105,34,0.446870,0.740536,0.341058,0.407407,0.5,0.477413,0.819921,0.392302,0.580247,0.6,41.7,-1600.0,-1238.60,-1113.0,-1400.0,44.3,-13.0,-110.0,-14.00,-105.17,-13.0,-14.5,-101.0,-11

In [168]:
cols_to_drop = ['h_PTS', 'a_PTS', 'h_LOSSES', 'h_Spread', 'Total', 'a_Spread']
combined_df = combined_df.drop(columns=cols_to_drop)
combined_df

,YEAR,a_season_FG_PCT_avg,a_season_FT_PCT_avg,a_season_FG3_PCT_avg,a_WIN_PCT,a_WIN_PCT_LAST10,h_season_FG_PCT_avg,h_season_FT_PCT_avg,h_season_FG3_PCT_avg,h_WIN_PCT,h_WIN_PCT_LAST10,h_PercentBet_ML,h_Open_Line_ML,h_Average_Line_ML,h_Best_Line_ML,h_Worst_Line_ML,h_PercentBet_Spread,h_Open_Line_Spread,h_Open_Odds_Spread,h_Average_Line_Spread,h_Average_Odds_Spread,h_Best_Line_Spread,h_Worst_Line_Spread,h_Best_Odds_Spread,h_Worst_Odds_Spread,h_PercentBet_OU,h_Open_Line_OU,h_Open_Odds_OU,h_Average_Line_OU,h_Average_Odds_OU,h_Best_Line_OU,h_Worst_Line_OU,h_Best_Odds_OU,h_Worst_Odds_OU,h_Pts,a_PercentBet_ML,a_Open_Line_ML,a_Average_Line_ML,a_Best_Line_ML,a_Worst_Line_ML,a_PercentBet_Spread,a_Open_Line_Spread,a_Open_Odds_Spread,a_Average_Line_Spread,a_Average_Odds_Spread,a_Best_Line_Spread,a_Worst_Line_Spread,a_Best_Odds_Spread,a_Worst_Odds_Spread,a_PercentBet_OU,a_Open_Line_OU,a_Open_Odds_OU,a_Average_Odds_OU,a_Best_Odds_OU,a_Worst_Odds_OU,a_Pts,Worst_Line_OU,Best_Line_OU,Average_Line_OU,h_cover,over_cover
0,2012-2013,0.465846,0.804308,0.361782,NaN,NaN,0.448566,0.772697,0.339461,NaN,NaN,55.4,-260.0,-246.00,-235.0,-255.0,55.6,-5.0,-110.0,-5.83,-110.33,-5.0,-6.0,-105.0,-115.0,52.7,187.0,-110.0,190.33,-110.17,191.0,187.0,-106.0,-115.0,94.0,44.6,220.0,209.00,215.0,200.0,44.4,5.0,-110.0,5.83,-108.00,6.0,5.0,-105.0,-110.0,47.3,187.0,-110.0,-108.17,-104.0,-110.0,84.0,187.0,191.0,190.33,0,0
1,2012-2013,0.451780,0.751817,0.350976,NaN,NaN,0.471600,0.683800,0.320200,NaN,NaN,55.2,-260.0,-270.00,-260.0,-280.0,51.8,-7.5,-110.0,-6.58,-108.17,-6.0,-7.5,-105.0,-110.0,48.4,182.0,-110.0,186.58,-109.33,188.0,182.0,-105.0,-115.0,120.0,44.8,220.0,234.80,244.0,225.0,48.2,7.5,-110.0,6.58,-108.50,7.5,6.0,-101.0,-115.0,51.6,182.0,-110.0,-107.33,-104.0,-110.0,107.0,182.0,188.0,186.58,1,1
2,2012-2013,0.452739,0.786101,0.344986,NaN,NaN,0.480692,0.803808,0.365885,NaN,NaN,59.2,-425.0,-492.00,-460.0,-550.0,51.0,-9.5,-110.0,-8.75,-109.50,-8.5,-9.5,-105.0,-115.0,57.4,185.0,-110.0,186.00,-109.17,187.0,185.0,-105.0,-110.0,91.0,40.8,350.0,394.60,420.0,375.0,49.0,9.5,-110.0,8.75,-107.17,9.5,8.5,-103.0,-110.0,42.6,185.0,-110.0,-107.50,-100.0,-110.0,99.0,185.0,187.0,186.00,0,1
3,2012-2013,0.456184,0.779842,0.359789,NaN,NaN,0.454478,0.714000,0.349896,NaN,NaN,58.1,125.0,50.00,102.0,-102.0,48.5,4.0,-110.0,1.92,-110.50,4.0,1.0,-108.0,-115.0,46.5,188.0,-110.0,189.75,-108.50,190.5,188.0,-101.0,-110.0,88.0,41.9,-145.0,-117.75,-113.0,-120.0,51.5,-4.0,-110.0,-1.92,-107.83,-1.0,-4.0,-102.0,-110.0,53.5,188.0,-110.0,-109.83,-109.0,-110.0,90.0,188.0,190.5,189.75,0,0
4,2012-2013,0.446300,0.740671,0.340429,NaN,NaN,0.477140,0.743980,0.374400,NaN,NaN,46.6,-125.0,109.75,110.0,109.0,41.6,-1.5,-110.0,1.25,-107.83,2.0,-1.5,-102.0,-110.0,52.9,202.0,-110.0,197.83,-110.00,202.0,196.5,-110.0,-110.0,84.0,53.4,105.0,-127.50,-120.0,-130.0,58.4,1.5,-110.0,-1.25,-110.50,1.5,-2.0,-108.0,-115.0,47.1,202.0,-110.0,-108.33,-100.0,-110.0,75.0,196.5,202.0,197.83,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8605,2018-2019,0.476958,0.784458,0.358021,0.740741,0.5,0.454015,0.715500,0.348742,0.592593,0.5,43.8,148.0,176.20,181.0,175.0,37.7,2.0,-102.0,4.08,-105.83,4.5,2.0,-102.0,-110.0,57.6,230.0,-110.0,227.67,-106.17,230.0,227.0,-102.0,-110.0,116.0,56.2,-170.0,-202.40,-190.0,-210.0,62.3,-2.0,-123.0,-4.08,-110.00,-2.0,-4.5,-101.0,-123.0,42.4,230.0,-110.0,-108.83,-105.0,-115.0,127.0,227.0,230.0,227.67,1,1
8606,2018-2019,0.446870,0.740536,0.341058,0.407407,0.5,0.477413,0.819921,0.392302,0.580247,0.6,41.7,-1600.0,-1238.60,-1113.0,-1400.0,44.3,-13.0,-110.0,-14.00,-105.17,-13.0,-14.5,-101.0,-110.0,50.4,218.0,-110.0,218.42,-109.17,219.0,218.0,-105.0,-115.0,105.0,58.3,912.0,858.20,950.0,750.0,55.7,13.0,-110.0,14.00,-109.83,14.5,13.0,-105.0,-115.0,49.6,218.0,-110.0,-105.83,-105.0,-110.0,94.0,218.0,219.0,218.42,0,0
8607,2

In [170]:
def moneyline_to_probability(odds):
    """Convert moneyline odds to implied probability (vectorized for arrays)."""
    return np.where(
        odds < 0,
        -odds / (-odds + 100),
        100 / (odds + 100)
    )

# Convert both home and away moneyline odds to implied probabilities
home_probs = moneyline_to_probability(combined_df['h_Average_Line_ML'])
away_probs = moneyline_to_probability(combined_df['a_Average_Line_ML'])

# Normalize to remove the vig (so that home + away = 1)
total_probs = home_probs + away_probs
combined_df['h_Implied_Prob_ML'] = home_probs / total_probs

In [171]:
if combined_df['h_PercentBet_ML'].max() > 1:
    combined_df['h_PercentBet_ML'] = combined_df['h_PercentBet_ML'] / 100.0
    
eps = 1e-6
p_public = combined_df['h_PercentBet_ML'].clip(eps, 1 - eps)
p_vegas = combined_df['h_Implied_Prob_ML'].clip(eps, 1 - eps)

# Log-odds based favorability
combined_df['h_PublicFavorability'] = np.log(p_public / (1 - p_public)) - np.log(p_vegas / (1 - p_vegas))

In [172]:
combined_df['h_PublicFavorability'].mean()

np.float64(1.4170858397529191)

In [190]:
combined_df

,YEAR,a_season_FG_PCT_avg,a_season_FT_PCT_avg,a_season_FG3_PCT_avg,a_WIN_PCT,a_WIN_PCT_LAST10,h_season_FG_PCT_avg,h_season_FT_PCT_avg,h_season_FG3_PCT_avg,h_WIN_PCT,h_WIN_PCT_LAST10,h_PercentBet_ML,h_Open_Line_ML,h_Average_Line_ML,h_Best_Line_ML,h_Worst_Line_ML,h_PercentBet_Spread,h_Open_Line_Spread,h_Open_Odds_Spread,h_Average_Line_Spread,h_Average_Odds_Spread,h_Best_Line_Spread,h_Worst_Line_Spread,h_Best_Odds_Spread,h_Worst_Odds_Spread,h_PercentBet_OU,h_Open_Line_OU,h_Open_Odds_OU,h_Average_Line_OU,h_Average_Odds_OU,h_Best_Line_OU,h_Worst_Line_OU,h_Best_Odds_OU,h_Worst_Odds_OU,a_PercentBet_ML,a_Open_Line_ML,a_Average_Line_ML,a_Best_Line_ML,a_Worst_Line_ML,a_PercentBet_Spread,a_Open_Line_Spread,a_Open_Odds_Spread,a_Average_Line_Spread,a_Average_Odds_Spread,a_Best_Line_Spread,a_Worst_Line_Spread,a_Best_Odds_Spread,a_Worst_Odds_Spread,a_PercentBet_OU,a_Open_Line_OU,a_Open_Odds_OU,a_Average_Odds_OU,a_Best_Odds_OU,a_Worst_Odds_OU,Worst_Line_OU,Best_Line_OU,Average_Line_OU,h_cover,h_Implied_Prob_ML,h_PublicFavorability
0,2012-2013,0.465846,0.804308,0.361782,NaN,NaN,0.448566,0.772697,0.339461,NaN,NaN,0.554,-260.0,-246.00,-235.0,-255.0,55.6,-5.0,-110.0,-5.83,-110.33,-5.0,-6.0,-105.0,-115.0,52.7,187.0,-110.0,190.33,-110.17,191.0,187.0,-106.0,-115.0,44.6,220.0,209.00,215.0,200.0,44.4,5.0,-110.0,5.83,-108.00,6.0,5.0,-105.0,-110.0,47.3,187.0,-110.0,-108.17,-104.0,-110.0,187.0,191.0,190.33,0,0.687201,-0.570218
1,2012-2013,0.451780,0.751817,0.350976,NaN,NaN,0.471600,0.683800,0.320200,NaN,NaN,0.552,-260.0,-270.00,-260.0,-280.0,51.8,-7.5,-110.0,-6.58,-108.17,-6.0,-7.5,-105.0,-110.0,48.4,182.0,-110.0,186.58,-109.33,188.0,182.0,-105.0,-115.0,44.8,220.0,234.80,244.0,225.0,48.2,7.5,-110.0,6.58,-108.50,7.5,6.0,-101.0,-115.0,51.6,182.0,-110.0,-107.33,-104.0,-110.0,182.0,188.0,186.58,1,0.709567,-0.684527
2,2012-2013,0.452739,0.786101,0.344986,NaN,NaN,0.480692,0.803808,0.365885,NaN,NaN,0.592,-425.0,-492.00,-460.0,-550.0,51.0,-9.5,-110.0,-8.75,-109.50,-8.5,-9.5,-105.0,-115.0,57.4,185.0,-110.0,186.00,-109.17,187.0,185.0,-105.0,-110.0,40.8,350.0,394.60,420.0,375.0,49.0,9.5,-110.0,8.75,-107.17,9.5,8.5,-103.0,-110.0,42.6,185.0,-110.0,-107.50,-100.0,-110.0,185.0,187.0,186.00,0,0.804325,-1.041312
3,2012-2013,0.456184,0.779842,0.359789,NaN,NaN,0.454478,0.714000,0.349896,NaN,NaN,0.581,125.0,50.00,102.0,-102.0,48.5,4.0,-110.0,1.92,-110.50,4.0,1.0,-108.0,-115.0,46.5,188.0,-110.0,189.75,-108.50,190.5,188.0,-101.0,-110.0,41.9,-145.0,-117.75,-113.0,-120.0,51.5,-4.0,-110.0,-1.92,-107.83,-1.0,-4.0,-102.0,-110.0,53.5,188.0,-110.0,-109.83,-109.0,-110.0,188.0,190.5,189.75,0,0.552139,0.117561
4,2012-2013,0.446300,0.740671,0.340429,NaN,NaN,0.477140,0.743980,0.374400,NaN,NaN,0.466,-125.0,109.75,110.0,109.0,41.6,-1.5,-110.0,1.25,-107.83,2.0,-1.5,-102.0,-110.0,52.9,202.0,-110.0,197.83,-110.00,202.0,196.5,-110.0,-110.0,53.4,105.0,-127.50,-120.0,-130.0,58.4,1.5,-110.0,-1.25,-110.50,1.5,-2.0,-108.0,-115.0,47.1,202.0,-110.0,-108.33,-100.0,-110.0,196.5,202.0,197.83,1,0.459660,0.025502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8605,2018-2019,0.476958,0.784458,0.358021,0.740741,0.5,0.454015,0.715500,0.348742,0.592593,0.5,0.438,148.0,176.20,181.0,175.0,37.7,2.0,-102.0,4.08,-105.83,4.5,2.0,-102.0,-110.0,57.6,230.0,-110.0,227.67,-106.17,230.0,227.0,-102.0,-110.0,56.2,-170.0,-202.40,-190.0,-210.0,62.3,-2.0,-123.0,-4.08,-110.00,-2.0,-4.5,-101.0,-123.0,42.4,230.0,-110.0,-108.83,-105.0,-115.0,227.0,230.0,227.67,1,0.351045,0.365167
8606,2018-2019,0.446870,0.740536,0.341058,0.407407,0.5,0.477413,0.819921,0.392302,0.580247,0.6,0.417,-1600.0,-1238.60,-1113.0,-1400.0,44.3,-13.0,-110.0,-14.00,-105.17,-13.0,-14.5,-101.0,-110.0,50.4,218.0,-110.0,218.42,-109.17,219.0,218.0,-105.0,-115.0,58.3,912.0,858.20,950.0,750.0,55.7,13.0,-110.0,14.00,-109.83,14.5,13.0,-105.0,-115.0,49.6,218.0,-110.0,-105.83,

In [ ]:
combined_df.drop(columns=['h_Pts', 'a_Pts', 'over_cover'], inplace=True)

In [191]:
import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch

# STEP 1: Split into train/val/test based on YEAR
train_years = ['2012-2013', '2013-2014', '2014-2015', '2015-2016']
val_years = ['2016-2017']
test_years = ['2017-2018', '2018-2019']

df_train = combined_df[combined_df['YEAR'].isin(train_years)].copy()
df_val = combined_df[combined_df['YEAR'].isin(val_years)].copy()
df_test = combined_df[combined_df['YEAR'].isin(test_years)].copy()

# CHECK: Are the splits roughly expected sizes?
print(f"Train shape: {df_train.shape}")
print(f"Val shape:   {df_val.shape}")
print(f"Test shape:  {df_test.shape}")


Train shape: (4920, 60)
Val shape:   (1230, 60)
Test shape:  (2460, 60)


In [192]:
# STEP 2: Drop YEAR, and all columns with 'OU' (except average), and lines/odds
columns_to_drop = ['YEAR']
columns_to_drop += [col for col in combined_df.columns if 'OU' in col and col not in ['h_Average_Line_OU', 'a_Average_Odds_OU']]
columns_to_drop += [col for col in combined_df.columns if ('Line' in col or 'Odds' in col) and col not in ['h_Average_Line_OU', 'a_Average_Odds_OU']]
columns_to_drop = list(set(columns_to_drop))  # remove duplicates

df_train.drop(columns=columns_to_drop, inplace=True, errors='ignore')
df_val.drop(columns=columns_to_drop, inplace=True, errors='ignore')
df_test.drop(columns=columns_to_drop, inplace=True, errors='ignore')


# CHECK: Remaining columns
print("Remaining columns:", df_train.columns.tolist())


Remaining columns: ['a_season_FG_PCT_avg', 'a_season_FT_PCT_avg', 'a_season_FG3_PCT_avg', 'a_WIN_PCT', 'a_WIN_PCT_LAST10', 'h_season_FG_PCT_avg', 'h_season_FT_PCT_avg', 'h_season_FG3_PCT_avg', 'h_WIN_PCT', 'h_WIN_PCT_LAST10', 'h_PercentBet_ML', 'h_PercentBet_Spread', 'h_Average_Line_OU', 'a_PercentBet_ML', 'a_PercentBet_Spread', 'a_Average_Odds_OU', 'h_cover', 'h_Implied_Prob_ML', 'h_PublicFavorability']


In [193]:
# STEP 3: Drop NaNs
df_train.dropna(inplace=True)
df_val.dropna(inplace=True)
df_test.dropna(inplace=True)

# Separate target from predictors
target_col = 'h_cover'
X_train = df_train.drop(columns=[target_col])
y_train = df_train[target_col].astype(int)

X_val = df_val.drop(columns=[target_col])
y_val = df_val[target_col].astype(int)

X_test = df_test.drop(columns=[target_col])
y_test = df_test[target_col].astype(int)

# CHECK: Ensure no missing values or object columns remain
print("Any NaNs in train?", X_train.isna().any().any())
print("Object cols in train:", X_train.select_dtypes(include='object').columns.tolist())


Any NaNs in train? False
Object cols in train: []


In [194]:

# Final defensive conversion to ensure clean float32 arrays
X_train_np = np.array(X_train, dtype=np.float32)
X_val_np = np.array(X_val, dtype=np.float32)
X_test_np = np.array(X_test, dtype=np.float32)

# Convert labels to int64 tensors (required by TabNet internally)
y_train = np.array(y_train).astype(np.int64)
y_val = np.array(y_val).astype(np.int64)
y_test = np.array(y_test).astype(np.int64)

# Create and train the model
tabnet_model = TabNetClassifier(
    device_name='cuda' if torch.cuda.is_available() else 'cpu',
    seed=42,
    verbose=1
)

tabnet_model.fit(
    X_train=X_train_np,
    y_train=y_train,
    eval_set=[(X_val_np, y_val)],
    eval_name=['val'],
    eval_metric=['accuracy'],
    max_epochs=200,
    patience=20,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0
)

/Users/lathangregg/Documents/MSDS/Spring25/DS6050_DL/Project/DL_project/.venv/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


RuntimeError: Could not infer dtype of numpy.float32

In [188]:

# Helper: encode categorical and convert to float32
def prepare_tabnet_input(df):
    df = df.copy()
    for col in df.columns:
        if df[col].dtype == 'object' or df[col].dtype.name == 'category':
            df[col] = LabelEncoder().fit_transform(df[col].astype(str))
    return df.astype(np.float32)

# Apply to all X sets
X_train_np = prepare_tabnet_input(X_train).to_numpy(dtype=np.float32)
X_val_np = prepare_tabnet_input(X_val).to_numpy(dtype=np.float32)
X_test_np = prepare_tabnet_input(X_test).to_numpy(dtype=np.float32)

# ✅ CHECK: Final dtype confirmation
print("Train dtype:", X_train_np.dtype, "| Shape:", X_train_np.shape)
print("Val dtype:", X_val_np.dtype)


Train dtype: float32 | Shape: (4844, 18)
Val dtype: float32


In [189]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score
import torch

# STEP 5: Fit TabNet
tabnet_model = TabNetClassifier(
    device_name='cuda' if torch.cuda.is_available() else 'cpu',
    verbose=1,
    seed=42
)

tabnet_model.fit(
    X_train=X_train_np, y_train=y_train,
    eval_set=[(X_val_np, y_val)],
    eval_name=['val'],
    eval_metric=['accuracy'],
    max_epochs=200,
    patience=20,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
)

# CHECK: Should see training logs, no dtype errors


/Users/lathangregg/Documents/MSDS/Spring25/DS6050_DL/Project/DL_project/.venv/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


RuntimeError: Could not infer dtype of numpy.float32

In [ ]:
# get rid of inverse columns 

### Setting up Data for the Model

In [61]:
# Adding season columns (ex: Season 2012-2013 is season 1)
season_order = sorted(combined_df['YEAR_x'].unique())
season_to_index = {season: i+1 for i, season in enumerate(season_order)}
combined_df['SEASON_NUM'] = combined_df['YEAR_x'].map(season_to_index)

# Dropping rows with Missing values for now
combined_df = combined_df.dropna()

In [62]:
# Filtering data into train (2012 - 2016), validation (2017), test seasons (2018-2019)
train_df = combined_df[combined_df['SEASON_NUM'] <= 3]    # Seasons 1-3
val_df = combined_df[combined_df['SEASON_NUM'] == 4]      # Season 4
test_df = combined_df[combined_df['SEASON_NUM'] >= 5]     # Seasons 5-6

In [66]:
combined_df.columns

Index(['GAME_ID', 'GAME_SEQUENCE', 'Date_x', 'YEAR_x', 'GAME_DATE_EST',
       'a_index_x', 'a_TEAM_ID', 'a_TEAM_WINS_LOSSES', 'a_PTS', 'a_FG_PCT',
       ...
       'h_Average_Line_OU', 'h_Average_Odds_OU', 'h_Best_Line_OU',
       'h_Worst_Line_OU', 'h_Best_Odds_OU', 'h_Worst_Odds_OU', 'h_Pts',
       'h_Spread', 'h_Result', 'h_Total'],
      dtype='object', length=166)

In [67]:
# Dropping columns that we don't need?
columns_to_drop = ['a_PTS', 'a_FG_PCT', 'a_FT_PCT','a_FG3_PCT', 'a_AST', 'a_REB', 'a_TOV','GAME_ID',
                   'h_PTS', 'h_FG_PCT', 'h_FT_PCT', 'h_FG3_PCT','h_AST', 'h_REB', 'h_TOV',
                   'GAME_SEQUENCE', 'Date_x', 'YEAR_x', 'a_index_x', 'a_index_y', 'a_TEAM_ID','h_TEAM_ID', 'a_TEAM_WINS_LOSSES',
                  'h_TEAM_WINS_LOSSES', 'IS_PLAYOFF_GAME', 'GAME_DATE_EST', 'h_index_x', 'h_index_y','SEASON_NUM', 'HOME_WIN']


# Creating train, validation, and test sets
X_train = train_df.drop(columns_to_drop, axis = 1)
y_train = train_df['HOME_WIN']

X_val = val_df.drop(columns_to_drop, axis = 1)
y_val = val_df['HOME_WIN']

X_test = test_df.drop(columns_to_drop, axis = 1)
y_test = test_df['HOME_WIN']

In [ ]:
X_train.drop(columns=['a_WINS', 'a_LOSSES', 'h_WINS', 'a_SEASON_GAMES_PLAYED', 'h_SEASON_GAMES_PLAYED', 'a_season_PTS_lag', 'a_season_AST_lag', 'a_season_REB_lag', 'a_season_TOV_lag', 'h_season_PTS_lag', 'h_season_AST_lag', 'h_season_REB_lag', 'h_season_TOV_lag'], inplace=True)

,a_WINS,a_LOSSES,h_WINS,h_LOSSES,a_SEASON_GAMES_PLAYED,h_SEASON_GAMES_PLAYED,a_season_PTS_lag,a_season_AST_lag,a_season_REB_lag,a_season_TOV_lag,a_season_FG_PCT_avg,a_season_FT_PCT_avg,a_season_FG3_PCT_avg,a_WIN_PCT,a_WIN_PCT_LAST10,h_season_PTS_lag,h_season_AST_lag,h_season_REB_lag,h_season_TOV_lag,h_season_FG_PCT_avg,h_season_FT_PCT_avg,h_season_FG3_PCT_avg,h_WIN_PCT,h_WIN_PCT_LAST10,a_Location,a_Team,a_OppTeam,a_TeamId,a_PercentBet_ML,a_Open_Line_ML,a_Pinnacle_ML,a_5dimes_ML,a_Heritage_ML,a_Bovada_ML,a_Betonline_ML,a_Average_Line_ML,a_Best_Line_ML,a_Worst_Line_ML,a_PercentBet_Spread,a_Open_Line_Spread,a_Open_Odds_Spread,a_Pinnacle_Line_Spread,a_Pinnacle_Odds_Spread,a_5dimes_Line_Spread,a_5dimes_Odds_Spread,a_Heritage_Line_Spread,a_Heritage_Odds_Spread,a_Bovada_Line_Spread,a_Bovada_Odds_Spread,a_Betonline_Line_Spread,a_Betonline_Odds_Spread,a_Average_Line_Spread,a_Average_Odds_Spread,a_Best_Line_Spread,a_Worst_Line_Spread,a_Best_Odds_Spread,a_Worst_Odds_Spread,a_PercentBet_OU,a_Open_Line_OU,a_Open_Odds_OU,a_Pinnacle_Line_OU,a_Pinnacle_Odds_OU,a_5dimes_Line_OU,a_5dimes_Odds_OU,a_Heritage_Line_OU,a_Heritage_Odds_OU,a_Bovada_Line_OU,a_Bovada_Odds_OU,a_Betonline_Line_OU,a_Betonline_Odds_OU,a_Average_Line_OU,a_Average_Odds_OU,a_Best_Line_OU,a_Worst_Line_OU,a_Best_Odds_OU,a_Worst_Odds_OU,a_Pts,a_Spread,a_Result,a_Total,h_Location,h_Team,h_OppTeam,h_TeamId,h_PercentBet_ML,h_Open_Line_ML,h_Pinnacle_ML,h_5dimes_ML,h_Heritage_ML,h_Bovada_ML,h_Betonline_ML,h_Average_Line_ML,h_Best_Line_ML,h_Worst_Line_ML,h_PercentBet_Spread,h_Open_Line_Spread,h_Open_Odds_Spread,h_Pinnacle_Line_Spread,h_Pinnacle_Odds_Spread,h_5dimes_Line_Spread,h_5dimes_Odds_Spread,h_Heritage_Line_Spread,h_Heritage_Odds_Spread,h_Bovada_Line_Spread,h_Bovada_Odds_Spread,h_Betonline_Line_Spread,h_Betonline_Odds_Spread,h_Average_Line_Spread,h_Average_Odds_Spread,h_Best_Line_Spread,h_Worst_Line_Spread,h_Best_Odds_Spread,h_Worst_Odds_Spread,h_PercentBet_OU,h_Open_Line_OU,h_Open_Odds_OU,h_Pinnacle_Line_OU,h_Pinnacle_Odds_OU,h_5dimes_Line_OU,h_5dimes_Odds_OU,h_Heritage_Line_OU,h_Heritage_Odds_OU,h_Bovada_Line_OU,h_Bovada_Odds_OU,h_Betonline_Line_OU,h_Betonline_Odds_OU,h_Average_Line_OU,h_Average_Odds_OU,h_Best_Line_OU,h_Worst_Line_OU,h_Best_Odds_OU,h_Worst_Odds_OU,h_Pts,h_Spread,h_Result,h_Total
0,1,0,1,0,1,1,94,22,54,20,0.456000,0.682000,0.350000,1.000000,1.0,0,0,0,0,0.489000,0.867167,0.401333,1.000000,1.0,home,Cleveland,Chicago,1.610613e+09,34.7,125.0,120.0,130.0,125.0,130.0,125.0,126.0,130.0,120.0,39.0,2.0,-110.0,2.5,-107.0,3.0,-105.0,2.5,-105.0,3.0,-105.0,3.0,-115.0,2.67,-107.83,3.0,2.0,-105.0,-115.0,63.2,186.0,-110.0,183.5,-105.0,183.5,-105.0,183.5,-110.0,184.0,-110.0,183.5,-110.0,184.00,-108.33,186.0,183.5,-105.0,-110.0,86.0,-29.0,L,201.0,away,Chicago,Cleveland,1.610613e+09,65.3,-145.0,-133.0,-140.0,-145.0,-150.0,-145.0,-142.6,-133.0,-150.0,61.0,-2.0,-110.0,-2.5,-103.0,-3.0,-105.0,-2.5,-115.0,-3.0,-115.0,-3.0,-105.0,-2.67,-108.83,-2.0,-3.0,-103.0,-115.0,36.8,186.0,-110.0,183.5,-105.0,183.5,-105.0,183.5,-110.0,184.0,-110.0,183.5,-110.0,184.00,-108.33,186.0,183.5,-105.0,-110.0,115.0,29.0,W,201.0
1,0,1,1,0,1,1,95,26,43,13,0.438000,0.913000,0.235000,0.000000,0.0,113,24,61,14,0.427000,0.738000,0.429000,1.000000,1.0,home,New Orleans,Utah,1.610613e+09,49.4,145.0,137.0,143.0,140.0,140.0,140.0,140.0,143.0,137.0,60.0,3.5,-110.0,3.5,-110.0,3.5,-105.0,3.5,-110.0,3.5,-110.0,3.5,-110.0,3.50,-109.17,3.5,3.5,-105.0,-110.0,50.0,189.5,-110.0,191.0,-102.0,191.0,-102.0,191.0,-110.0,191.0,-110.0,191.0,-110.0,190.75,-107.33,191.0,189.5,-102.0,-110.0,88.0,2.0,W,174.0,away,Utah,New Orleans,1.610613e+09,50.6,-165.0,-152.0,-153.0,-160.0,-160.0,-160.0,-157.0,-152.0,-160.0,40.0,-3.5,-110.0,-3.5,-100.0,-3.5,-105.0,-3.5,-110.0,-3.5,-110.0,-3.5,-110.0,-3.50,-107.50,-3.5,-3.5,-100.0,-110.0,50.0,189.5,-110.0,191.0,-108.0,191.0,-108.0,191.0,-110.0,191.0,-110.0,191.0,-110.0,190.75,-109.33,191.0,189.5,-108.0,-110.0,86.0,-2.0,L,174.0
2,1,0,0,1,1,1,0,0,0,0,0.460900,0.793333,0.321033,1.000000,1.0,84,18,48,18,0

In [39]:
# Checking Nans totals for differnet columns
vegas.isna().sum().sort_values(ascending=False)

a_Bovada_ML        1685
h_Bovada_ML        1685
a_Open_Line_ML      168
h_Open_Line_ML      168
a_Betonline_ML      144
                   ... 
h_PercentBet_OU       0
h_Pts                 0
h_Spread              0
h_Result              0
h_Total               0
Length: 117, dtype: int64

### Feature Engineering:

In [40]:
from sklearn.preprocessing import StandardScaler

In [41]:
# Scaling the data with mean 0 and standard deviation 1

# Fit scaler on training features only
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Apply same scaler to val/test
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [47]:
X_test_scaled.dtype

dtype('float64')

In [44]:
# Testing TabNet Model
clf = TabNetClassifier(
    n_d=16,               # Number of decision prediction steps
    n_a=16,               # Number of attention steps (usually equal to n_d)
    n_steps=5,            # Number of steps in TabNet
    gamma=1.5,            # Relaxation factor
    lambda_sparse=1e-4,   # Sparse regularization
    optimizer_fn=torch.optim.Adam,  
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"step_size":10, "gamma":0.9},  
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    verbose=1,
    seed=42
)

clf.fit(
    X_train=X_train_scaled, y_train=y_train,
    eval_set=[(X_val_scaled, y_val)],
    eval_name=["val"],
    eval_metric=["accuracy"],
    max_epochs=100,
    patience=10,         # Early stopping
    batch_size=1024, 
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

/Users/lathangregg/Documents/MSDS/Spring25/DS6050_DL/Project/DL_project/.venv/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


RuntimeError: Could not infer dtype of numpy.float32

In [74]:
# Predicting on test data
y_pred = clf.predict(X_test_scaled)
y_proba = clf.predict_proba(X_test_scaled)[:, 1]

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))

Accuracy: 0.6057111477210324
ROC AUC: 0.6376477429251237
              precision    recall  f1-score   support

           0       0.60      0.63      0.61      1801
           1       0.62      0.58      0.60      1841

    accuracy                           0.61      3642
   macro avg       0.61      0.61      0.61      3642
weighted avg       0.61      0.61      0.61      3642

